In [ ]:
import numpy as np
import sys
import cv2

import tensorflow as tf

# settings
output_model = "7seg.tflite"
input_model = '7seg.h5'
# load validation set
import glob

image_folder = "calib_img"
img_path = glob.glob(image_folder+"/*.png")
if len(img_path)==0:
    print("image not found")
    # sys.exit(1)

validation_data_set=[]
for file_name in img_path:
    img = cv2.imread(file_name) #BGR
    img = cv2.resize(img,(28, 28))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = img/255.0
    # img = np.asarray(img)
    img = np.asarray(img, dtype=np.float32)
    img = np.expand_dims(img, axis=0)
    img = np.expand_dims(img, axis=3)

    # mean = np.asarray([103.939, 116.779, 123.68], dtype=np.float32)
    # ary = ary - mean
    # ary = np.minimum(ary,127)
    # ary = np.maximum(ary,-128)
    #print(ary.shape)
    validation_data_set.append(img)
# validation_data_set = validation_data_set.astype(np.uint8)
# validation_data_set = [np.uint8(item) for item in validation_data_set]

#quantize
def representative_dataset_gen():
  for i in range(len(validation_data_set)):
    yield [validation_data_set[i]]

# def representative_dataset_gen():
#    for i in range(len(validation_data_set)):
#       input_image = tf.cast(validation_data_set[i], tf.float32)
#       input_image = tf.reshape(input_image, [1,28,28,1])
#       yield ([input_image])
      # yield {"image":input_image}

keras_model = tf.keras.models.load_model(input_model)

converter = tf.lite.TFLiteConverter.from_keras_model(keras_model)
# converter = tf.lite.TFLiteConverter.from_saved_model(input_model)

#ダイナミック
tflite_quant_model = converter.convert()

#ダイナミック&最適化
# converter.optimizations = [tf.lite.Optimize.DEFAULT]
# tflite_quant_model = converter.convert()

#完全な整数
# converter.optimizations = [tf.lite.Optimize.DEFAULT]
# converter.representative_dataset = representative_dataset_gen
# converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
# converter.inference_input_type = tf.int8
# converter.inference_output_type = tf.int8
# tflite_quant_model = converter.convert()

#完全な整数量子化
# converter.experimental_new_converter = True   #<--- Tensorflow v2.2.x以降を使用している場合は不要
# converter.optimizations = [tf.lite.Optimize.DEFAULT]
## converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
# converter.inference_input_type = tf.uint8
# converter.inference_output_type = tf.uint8
# converter.representative_dataset = representative_dataset_gen
# tflite_quant_model = converter.convert()

with open(output_model, 'wb') as o_:
    o_.write(tflite_quant_model)

image not found
